In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision
import numpy as np
import json
import time
import sys
import copy

from kWTA import models
from kWTA import activation
from kWTA import attack
from kWTA import training
from kWTA import utilities
from kWTA import densenet
from kWTA import resnet
from kWTA import wideresnet

norm_mean = 0
norm_var = 1
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((norm_mean,norm_mean,norm_mean), (norm_var, norm_var, norm_var)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((norm_mean,norm_mean,norm_mean), (norm_var, norm_var, norm_var)),
])



In [3]:
svhn_train = datasets.SVHN("./data", split='train', download=True, transform=transform_train)
svhn_test = datasets.SVHN("./data", split='test', download=True, transform=transform_test)
train_loader = DataLoader(svhn_train, batch_size = 512, shuffle=True)
#test_loader = DataLoader(svhn_test, batch_size = 50, shuffle=True)

100%|███████████████████████████████████████████████████████████████▉| 182034432/182040794 [07:05<00:00, 423821.79it/s]


0it [00:00, ?it/s]
  6%|████                                                               | 3956736/64275384 [00:10<02:15, 444357.38it/s]


  8%|█████▋                                                             | 5439488/64275384 [00:13<02:36, 376165.01it/s]
182042624it [07:20, 423821.79it/s]                                                                                     
 13%|████████▌                                                          | 8265728/64275384 [00:20<02:15, 414155.46it/s]


 20%|████████████▉                                                     | 12640256/64275384 [00:30<02:00, 427300.97it/s]


 26%|█████████████████▍                                                | 17006592/64275384 [00:40<01:50, 429391.52it/s]


...

 74%|████████████████████████████████████████████████▊                 | 47587328/64275384 [01:52<00:39, 427436.36it/s]


 81%|█████████████████████████████████████████████████████▎            | 51953664/64275384 [02:02<00:28, 438032.70it/s]


 88%|█████████████████████████████████████████████████████████▊        | 56320000/64275384 [02:12<00:18, 432759.05it/s]


 94%|██████████████████████████████████████████████████████████████▎   | 60694528/64275384 [02:22<00:08, 430972.58it/s]


100%|█████████████████████████████████████████████████████████████████▉| 64225280/64275384 [02:30<00:00, 422472.04it/s]

## Regular training

### ReLU Resnet18

In [ ]:
device = torch.device('cuda:0')
model = resnet.ResNet18().to(device)
opt = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
eps = 0.047
for ep in range(80):
    if ep == 50:
        for param_group in opt.param_groups:
                param_group['lr'] = 0.01
    train_err, train_loss = training.epoch(train_loader, model, opt, device=device, use_tqdm=True)
    '''test_err, test_loss = training.epoch(test_loader, model, device=device, use_tqdm=True)
    adv_err, adv_loss = training.epoch_adversarial(test_loader,
        model, attack=attack.pgd_linf_untargeted, device=device, num_iter=20, 
        use_tqdm=True, epsilon=eps, randomize=True, alpha=0.004, n_test=1000)
    print('epoch', ep, 'train err', train_err, 'test err', test_err, 'adv_err', adv_err)'''
    torch.save(model.state_dict(), 'models/resnet18_svhn.pth')

### kWTA-0.1 Resnet18

In [ ]:
device = torch.device('cuda:0')
model = resnet.SparseResNet18(sparsities=[0.1,0.1,0.1,0.1], sparse_func='vol').to(device)
opt = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
eps = 0.047
for ep in range(80):
    if ep == 50:
        for param_group in opt.param_groups:
                param_group['lr'] = 0.01
    train_err, train_loss = training.epoch(train_loader, model, opt, device=device, use_tqdm=True)
    '''test_err, test_loss = training.epoch(test_loader, model, device=device, use_tqdm=True)
    adv_err, adv_loss = training.epoch_adversarial(test_loader,
        model, attack=attack.pgd_linf_untargeted, device=device, num_iter=20, 
        use_tqdm=True, epsilon=eps, randomize=True, alpha=0.004, n_test=1000)
    print('epoch', ep, 'train err', train_err, 'test err', test_err, 'adv_err', adv_err)'''
    torch.save(model.state_dict(), 'models/spresnet18_0.1_svhn.pth')

### kWTA-0.2 Resnet18

In [ ]:
device = torch.device('cuda:0')
model = resnet.SparseResNet18(sparsities=[0.2,0.2,0.2,0.2], sparse_func='vol').to(device)
opt = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
eps = 0.047
for ep in range(80):
    if ep == 50:
        for param_group in opt.param_groups:
                param_group['lr'] = 0.01
    train_err, train_loss = training.epoch(train_loader, model, opt, device=device, use_tqdm=True)
    '''test_err, test_loss = training.epoch(test_loader, model, device=device, use_tqdm=True)
    adv_err, adv_loss = training.epoch_adversarial(test_loader,
        model, attack=attack.pgd_linf_untargeted, device=device, num_iter=20, 
        use_tqdm=True, epsilon=eps, randomize=True, alpha=0.004, n_test=1000)
    print('epoch', ep, 'train err', train_err, 'test err', test_err, 'adv_err', adv_err)'''
    torch.save(model.state_dict(), 'models/spresnet18_0.2_svhn.pth')

## Adv Training

### ReLU Resnet18

In [ ]:
device = torch.device('cuda:0')
model = resnet.ResNet18().to(device)
opt = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
eps = 0.047
for ep in range(80):
    if ep == 50:
        for param_group in opt.param_groups:
                param_group['lr'] = 0.01
    train_err, train_loss = training.epoch_adversarial(train_loader, model, opt=opt,
            attack=attack.pgd_linf_untargeted, device=device, num_iter=10, 
        use_tqdm=True, epsilon=eps, randomize=True, alpha=0.007)
    '''test_err, test_loss = training.epoch(test_loader, model, device=device, use_tqdm=True)
    adv_err, adv_loss = training.epoch_adversarial(test_loader,
        model, attack=attack.pgd_linf_untargeted, device=device, num_iter=20, 
        use_tqdm=True, epsilon=eps, randomize=True, alpha=0.004, n_test=1000)
    print('epoch', ep, 'train err', train_err, 'test err', test_err, 'adv_err', adv_err)'''
    torch.save(model.state_dict(), 'models/resnet18_svhn_adv.pth')

### kWTA-0.1 Resnet18

In [ ]:
device = torch.device('cuda:0')
model = resnet.SparseResNet18(sparsities=[0.1,0.1,0.1,0.1], sparse_func='vol').to(device)
opt = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
eps = 0.047
for ep in range(80):
    if ep == 50:
        for param_group in opt.param_groups:
                param_group['lr'] = 0.01
    train_err, train_loss = training.epoch_adversarial(train_loader, model, opt=opt,
            attack=attack.pgd_linf_untargeted, device=device, num_iter=10, 
        use_tqdm=True, epsilon=eps, randomize=True, alpha=0.007)
    '''test_err, test_loss = training.epoch(test_loader, model, device=device, use_tqdm=True)
    adv_err, adv_loss = training.epoch_adversarial(test_loader,
        model, attack=attack.pgd_linf_untargeted, device=device, num_iter=20, 
        use_tqdm=True, epsilon=eps, randomize=True, alpha=0.004, n_test=1000)
    print('epoch', ep, 'train err', train_err, 'test err', test_err, 'adv_err', adv_err)'''
    torch.save(model.state_dict(), 'models/spresnet18_0.1_svhn_adv.pth')

### kWTA-0.2 Resnet18

In [ ]:
device = torch.device('cuda:0')
model = resnet.SparseResNet18(sparsities=[0.2,0.2,0.2,0.2], sparse_func='vol').to(device)
opt = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
eps = 0.047
for ep in range(80):
    if ep == 50:
        for param_group in opt.param_groups:
                param_group['lr'] = 0.01
    train_err, train_loss = training.epoch_adversarial(train_loader, model, opt=opt,
            attack=attack.pgd_linf_untargeted, device=device, num_iter=10, 
        use_tqdm=True, epsilon=eps, randomize=True, alpha=0.007)
    '''test_err, test_loss = training.epoch(test_loader, model, device=device, use_tqdm=True)
    adv_err, adv_loss = training.epoch_adversarial(test_loader,
        model, attack=attack.pgd_linf_untargeted, device=device, num_iter=20, 
        use_tqdm=True, epsilon=eps, randomize=True, alpha=0.004, n_test=1000)
    print('epoch', ep, 'train err', train_err, 'test err', test_err, 'adv_err', adv_err)'''
    torch.save(model.state_dict(), 'models/spresnet18_0.2_svhn_adv.pth')